# Key functions and tool used for ICEO BS analysis

**Goal**

- Load the data
- Compute the LoC and RoC 
- Visualize LoC and RoC on a spectrogram



In [ ]:
import glob
import math
import os
import mne
import numpy as np
import pandas as pd
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.interpolate import interp1d
from scipy.signal import find_peaks, spectrogram
from scipy.optimize import least_squares
from sklearn.mixture import GaussianMixture

from boost_loc_roc.LoC_RoC import extract_loc_roc, Truncate_fif, plot_spectrogram


In [ ]:
# Here we load one EEG recording
filename = 'data/raw/eeg_example.fif'
file_tmp = mne.io.read_raw_fif(filename)

# LoC/RoC detection 


**Remark**
The following code is used on ICEO file but should be easily extended to other dataset. 

This step can be embedded directly in the loop dealing with other segmentation processes.

The following cell is thus only for pedagogical purposes, but can also serve as an inspection step. 

In [ ]:
%%time

file_tmp = Truncate_fif(file_tmp)
duration_sec = file_tmp.n_times / file_tmp.info['sfreq']
print('sfreq', file_tmp.info['sfreq'])

if duration_sec > 5 * 60:
    # Extract sampling frequency and make a time vector
    Fs = file_tmp.info['sfreq']  # Hz
    time = np.linspace(0, file_tmp.get_data().shape[1]/Fs, file_tmp.get_data().shape[1], endpoint=False)
    Fp2 = file_tmp.get_data()[1,:]*10**6  # 10*6 because .fif is saved in Volts
    
    # Compute LoC and RoC
    time_loc, time_roc, t_proba, proba = extract_loc_roc(file_tmp)

    # Visualize Spectrogram, EEG, and proba
    plot_spectrogram(time_loc, time_roc, Fp2, Fs, time, t_proba, proba)